In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
KEYWORDS = ['тест', 'истории']
page_link = 'https://habr.com/ru/all/'
base_link = 'https://habr.com'

def get_habr_articles(page_link):
    res = requests.get(page_link)
    soup = BeautifulSoup(res.text)
    articles = soup.findAll('div', 'tm-article-snippet')
    return articles

def get_articles_data(articles, base_link):
    df = pd.DataFrame()
    for article in articles:
        title = article.find('a', 'tm-article-snippet__title-link').find('span').text
        date = article.find('span', 'tm-article-snippet__datetime-published').find('time').get('datetime')
        link = base_link + article.find('a', 'tm-article-snippet__title-link').get('href')
        res_article = requests.get(link)
        soup_article = BeautifulSoup(res_article.text, features='lxml')
        text = soup_article.find('div', 'article-formatted-body').find('div').text
        row = {'date' : date, 'title' : title, 'link' : link, 'text' : text}
        df = pd.concat([df, pd.DataFrame([row])]).reset_index(drop=True)
    return df

def get_filtered_articles_data(df, keywords):
    fitered_df = pd.DataFrame()
    for keyword in keywords:
        fitered_df_by_one_keyword = df[df.title.str.contains(keyword) | df.text.str.contains(keyword)]
        fitered_df = pd.concat([fitered_df, fitered_df_by_one_keyword])
    return fitered_df.reset_index(drop=True)

articles = get_habr_articles(page_link)
df = get_articles_data(articles, base_link)
fitered_df = get_filtered_articles_data(df, KEYWORDS)

print(fitered_df)

                        date  \
0   2022-03-02T14:24:21.000Z   
1   2022-03-02T14:00:31.000Z   
2   2022-03-02T12:18:37.000Z   
3   2022-03-02T11:11:11.000Z   
4   2022-03-02T10:47:00.000Z   
5   2022-03-02T09:08:54.000Z   
6   2022-03-02T08:48:08.000Z   
7   2022-03-02T08:37:40.000Z   
8   2022-03-02T08:00:03.000Z   
9   2022-03-02T07:08:14.000Z   
10  2022-03-02T06:50:25.000Z   
11  2022-03-02T04:17:52.000Z   
12  2022-03-02T10:47:00.000Z   
13  2022-03-02T09:08:54.000Z   

                                                title  \
0     О Thread и ThreadPool в .NET подробно (часть 2)   
1     О Thread и ThreadPool в .NET подробно (часть 1)   
2               Не самые популярные методы Django ORM   
3           Недельный геймдев: #59 — 27 февраля, 2022   
4   Сочетание Shift-Left и «Традиционной» модели т...   
5   По новым рельсам: как Страховой Дом ВСК запуст...   
6   Приватность в электронной почте: нам нужно исп...   
7   Flutter: простыми словами про ассинхронность и...   
8   Об

In [4]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
EMAIL = ['xxx@x.ru', 'yyy@y.ru']
headers_ = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}
json_ = {
    'emailAddresses': EMAIL
}
res = requests.post(url, json=json_, headers=headers_).json()['breaches']
df2 = pd.DataFrame()
for (key, value) in res.items():
    row = {'data' : value['publishDate'], 'source' : value['site'], 'description' : value['description']}
    df2 = pd.concat([df2, pd.DataFrame([row])]).reset_index(drop=True)
print(df2)

                   data            source  \
0  2016-11-01T00:00:00Z            qip.ru   
1  2017-02-14T00:00:00Z    parapa.mail.ru   
2  2016-10-29T00:00:00Z            vk.com   
3  2016-10-21T00:00:00Z         adobe.com   
4  2017-02-14T00:00:00Z     cfire.mail.ru   
5  2017-01-31T00:00:00Z  cdprojektred.com   
6  2016-10-23T00:00:00Z         imesh.com   

                                         description  
0  In 2011, Russian instant messaging service pro...  
1  In July and August 2016, two criminals execute...  
2  Popular Russian social networking platform VKo...  
3  In October of 2013, criminals penetrated Adobe...  
4  In July and August of 2016, two criminals carr...  
5  In March 2016, CDProjektRed.com.com's forum da...  
6  In June 2016, a cache of over 51 million user ...  
